This example copies a lot from [a pytorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)


In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim


# To be able to import model
sys.path.append('/'+'/'.join(os.getcwd().split('/')[:-1])+'/')

from model.Conversion import Start,Stop
from model.activations import ReLU,PACT
from model.linear import Linear
from model.wrapped import FlattenM,MaxPool2d,Dropout
from model.blocks import ConvBnA,LinBnA
from model.sequential import Sequential
from model.Quantizer import LinQuantExpScale
from model.QuantizationMethods.MinMSE import MinMSE_convolution_weight_quantization, MinMSE_linear_weight_quantization

from tqdm import tqdm

path = 'cifa10_vgg'


if not os.path.exists('./runs'):
    os.mkdir('./runs')
if not os.path.exists(f'./runs/{path}'):
    os.mkdir(f'./runs/{path}')

In [2]:
batch_size = 80
epochs = 50

In [3]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (.3, .3, .3))])
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (.3, .3, .3))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
weight_quant = MinMSE_convolution_weight_quantization

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Start and Stop modules convert the Float values to the fake quantized domain
        # and during inference to the integer domain expressed by a float (necessary due to NVIDIA)
        self.start = Start(bits=8,size=(1,3,1,1),mode="auto",auto_runs=2)
        # mode="auto" simply measures min and max of the input and quantized to them in a symmetric manner 
        self.stop = Stop(size=(1,10))
        
        self.seq = Sequential(
            ConvBnA(  3, 64,3,1,1,activation=PACT,weight_quant=weight_quant),
            ConvBnA( 64, 64,3,1,1,activation=PACT,weight_quant=weight_quant),
            MaxPool2d(2,2),
            Dropout(0.1),
            ConvBnA( 64,128,3,1,1,activation=PACT,weight_quant=weight_quant),
            ConvBnA(128,128,3,1,1,activation=PACT,weight_quant=weight_quant),
            MaxPool2d(2,2),
            Dropout(0.1),
            ConvBnA(128,256,3,1,1,activation=PACT,weight_quant=weight_quant),
            ConvBnA(256,256,3,1,1,activation=PACT,weight_quant=weight_quant),
            Dropout(0.1),
            MaxPool2d(2,2),
            FlattenM(1),
            LinBnA(256*4*4,10,weight_quant=MinMSE_linear_weight_quantization,weight_quant_channel_wise=True,activation=LinQuantExpScale,affine=False),
        )

    def forward(self, x):
        x = self.start(x)
        x = self.seq(x)
        x = self.stop(x)
        return x

This example copies a lot from [a pytorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)


In [5]:
def eval(net, pr=True):
    global testloader,device
    global best
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device,non_blocking=True)
            labels = labels.to(device,non_blocking=True)
            # calculate outputs by running images through the network
            outputs = net(images)
            # loss = criterion(outputs, target)

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    net.train()
    if pr:
        print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:4.1f} %')
    if best < 100 * correct / total:
        best = 100 * correct / total

    return 100 * correct / total, best


In [6]:
net = Net()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)


criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9,weight_decay=5e-4)
sched= optim.lr_scheduler.CosineAnnealingLR(optimizer,epochs,1e-5,verbose=False)

best = 0

In [7]:
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    training_running_correct = 0
    with tqdm(enumerate(trainloader, 0),total=len(trainloader),disable=False) as t:
        for i, (data,target) in t:
            # get the inputs; data is a list of [inputs, labels]
            target = target.to(device,non_blocking=True)
            data = data.to(device,non_blocking=True)

            # zero the parameter gradients
            optimizer.zero_grad(True)

            # forward + backward + optimize
            outputs = net(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()


            # print statistics
            running_loss += loss.item()

            _, preds = torch.sort(outputs.data.clone().detach(), 1,descending=True)
            training_running_correct += (preds[:,0].view(-1,1) == target.view(-1,1)).any(dim=1).sum().detach().item()
            t.set_postfix({'loss': running_loss/(i+1),'acc':training_running_correct/((i+1)*batch_size)})
    ev = eval(net,pr=False)
    print(f'[{epoch + 1:3d}, {i + 1:5d}] loss: {running_loss*batch_size/len(trainloader):6.3f},Train Acc: {training_running_correct/len(trainloader):3.1f}, Test Acc:{ev[0]:3.1f}%, Best test Acc:{ev[1]:3.1f}%')
    running_loss = 0.0
    torch.save(net.state_dict(),f"./runs/{path}/ckp.pt")
    sched.step()
            
print('Finished Training')


100%|██████████| 625/625 [00:19<00:00, 32.50it/s, loss=1.53, acc=0.55] 


reduce autorun by 1: 1 min/max -1.6666666269302368 / 1.6666666269302368
[  1,   625] loss: 122.563,Train Acc: 44.0, Test Acc:64.4%, Best test Acc:64.4%


100%|██████████| 625/625 [00:17<00:00, 36.26it/s, loss=1.23, acc=0.722]


reduce autorun by 1: 0 min/max -1.6666666269302368 / 1.6666666269302368
[  2,   625] loss: 98.073,Train Acc: 57.8, Test Acc:72.1%, Best test Acc:72.1%


100%|██████████| 625/625 [00:17<00:00, 36.42it/s, loss=1.12, acc=0.777]


...

[ 48,   625] loss: 57.201,Train Acc: 77.4, Test Acc:90.6%, Best test Acc:90.6%


100%|██████████| 625/625 [00:14<00:00, 42.03it/s, loss=0.71, acc=0.969] 


[ 49,   625] loss: 56.834,Train Acc: 77.5, Test Acc:90.7%, Best test Acc:90.7%


100%|██████████| 625/625 [00:14<00:00, 41.97it/s, loss=0.71, acc=0.971] 


[ 50,   625] loss: 56.784,Train Acc: 77.6, Test Acc:90.7%, Best test Acc:90.7%
Finished Training
